### Explanation
This code identifies bacterial colonies from Petri dish time-lapse images and based on the fluorescent signal it is able to compute a characteristic pattern signal for each fluorescent protein. With this information, the code classify each of the identified colonies. 

##### Requirements
 - Image data
 - Have _fluopi_ module installed (open Terminal in Mac or command line in Windows and type  `pip install fluopi`; more info [here](https://packaging.python.org/tutorials/installing-packages/) about installing python packages)


## Analysis


### Loading packages

In [ ]:
# import some useful packages to manage the data

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os.path

# modify some matplotlib parameters to manage the images in illustrator
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

In [ ]:
# Import the FluoPi module
import fluopi.analysis as flua
import fluopi.plotting as flup

### Loading Data
In each _pathname_ is the complete path of the folder where images are stored. The example data can be obtained from the **_Files_** section in the [OSF FluoPi proyect](https://osf.io/dy6p2/). There you can download it from the *Google drive/Example_data* folder.

In [ ]:
pathname1 = os.path.abspath('Images/Classifier/sfGFP_data/')
#refered to google drive folder Example_data\Classifier\sfGFP_data`
                      
pathname2 = os.path.abspath('Images/Classifier/CyOFP_data/')
#refered to google drive folder Example_data\Classifier\CyOFP_data

pathname3 = os.path.abspath('Images/Classifier/BeRFP_data/')
#refered to google drive folder Example_data\Classifier\BeRFP_data

pathname4 =  os.path.abspath('Images/Classifier/RGO_data/')
#refered to google drive folder Example_data\Classifier\RGO_data

Then, make the image names:

In [ ]:
fname1 = os.path.join(pathname1, "image_%04d"+".jpg")
                      
fname2 = os.path.join(pathname2, "image_%04d"+".jpg")

fname3 = os.path.join(pathname3, "image_%04d"+".jpg")

fname4 =  os.path.join(pathname4, "image_%04d"+".jpg")

And also save the number name of the last image

In [ ]:
lim1=int(os.listdir(pathname1)[-1].split('_')[-1].split('.')[0])
lim2=int(os.listdir(pathname2)[-1].split('_')[-1].split('.')[0])
lim3=int(os.listdir(pathname3)[-1].split('_')[-1].split('.')[0])
lim4=int(os.listdir(pathname4)[-1].split('_')[-1].split('.')[0])

Let's check the data loading and channel structure (e.g. with the last frame image of the second data set)

In [ ]:
flup.plot_im_frame(fname2,lim2)

In [ ]:
flup.plt_im_frame_channels(fname2,lim2)

Count the files in each data set:

In [ ]:
imCount1=flua.count_files(pathname1,"jpg")
imCount2=flua.count_files(pathname2,"jpg")
imCount3=flua.count_files(pathname3,"jpg")
imCount4=flua.count_files(pathname4,"jpg")

Now, we organize the data and split it into the different channels. Also, here you can define with frames of the whole original data you are going to use (i.e. by defining `frame > 1` you can select which data to use)

In [ ]:
DataS1={}
frames1 = 10   # use one image every 10 frames
DataS1['R'],DataS1['G'],DataS1['B']=flua.get_im_data(frames1,imCount1,fname1)
DataS1['Im']=fname1     # to store the related image source

In [ ]:
DataS2={}
frames2 = 5     
DataS2['R'],DataS2['G'],DataS2['B']=flua.get_im_data(frames2,imCount2,fname2)
DataS2['Im']=fname2

In [ ]:
DataS3={}
frames3 = 5     
DataS3['R'],DataS3['G'],DataS3['B']=flua.get_im_data(frames3,imCount3,fname3) 
DataS3['Im']=fname3

In [ ]:
DataS4={}
frames4 = 5    
DataS4['R'],DataS4['G'],DataS4['B']=flua.get_im_data(frames4,imCount4,fname4)
DataS4['Im']=fname4

Now we have each data serie on a dictionary, organized in arrays for each channel.

### Background subtraction
First step is to identify a good background area and compute his value over time:

In [ ]:
BG1=flua.bg_value(200,370,10,400,DataS1,imCount1)

In [ ]:
BG2=flua.bg_value(180,360,420,750,DataS2,imCount2)

In [ ]:
BG3=flua.bg_value(100,380,50,430,DataS3,imCount3)

In [ ]:
BG4=flua.bg_value(450,670,260,420,DataS4,imCount4)

We can subtract this background vector value to each frame on the data to eliminate the background effect

In [ ]:
DataSB1=flua.bg_subst(DataS1,BG1)

In [ ]:
DataSB2=flua.bg_subst(DataS2,BG2)

In [ ]:
DataSB3=flua.bg_subst(DataS3,BG3)

In [ ]:
DataSB4=flua.bg_subst(DataS4,BG4)

### Get colony position
A good way to identify where colonies are situated is by summing the value of each pixel for all the channels and all the frames

In [ ]:
SumData1=flua.data_sum_time(DataSB1)

In [ ]:
SumData2=flua.data_sum_time(DataSB2)

In [ ]:
SumData3=flua.data_sum_time(DataSB3)

In [ ]:
SumData4=flua.data_sum_time(DataSB4)

####  Apply a filter to smooth the data 
This process let us reduce noise and improve border detection

In [ ]:
sDatS1,sDatSall1,SDatST1=flua.smooth_data(DataSB1,1.5)

In [ ]:
sDatS2,sDatSall2,SDatST2=flua.smooth_data(DataSB2,1.5)

In [ ]:
sDatS3,sDatSall3,SDatST3=flua.smooth_data(DataSB3,1.5)

In [ ]:
sDatS4,sDatSall4,SDatST4=flua.smooth_data(DataSB4,1.5)

### Blob detection

In [ ]:
#blobD1 = flua.colony_blobs_id(sDatS1['G'],0.35,DataS1['Im']%(lim1-1)) 
#for green it was better to use only the green channel

blobD1 = flua.colony_blobs_id(sDatS1['G'],0.35,DataS1['Im']%(lim1),filename='Plots/Classifier/Blobs_sfGFP') 

In [ ]:
#blobD2 = flua.colony_blobs_id(sDatSall2,0.34,DataS2['Im']%(lim2-1))
blobD2 = flua.colony_blobs_id(sDatSall2,0.34,DataS2['Im']%(lim2),filename='Plots/Classifier/Blobs_CyOFP')

In [ ]:
#blobD3 = flua.colony_blobs_id(sDatSall3,0.34,DataS3['Im']%(lim3-1))
blobD3 = flua.colony_blobs_id(sDatSall3,0.34,DataS3['Im']%(lim3),filename='Plots/Classifier/Blobs_BeRFP')

In [ ]:
#blobD4 = flua.colony_blobs_id(sDatSall4,0.37,DataS4['Im']%(lim4-1))
blobD4 = flua.colony_blobs_id(sDatSall4,0.37,DataS4['Im']%(lim4),filename='Plots/Classifier/Blobs_MixFPs')

In [ ]:
Rois1,RoisC1,NC1 = flua.obtain_rois(DataSB1,blobD1)
Rois2,RoisC2,NC2 = flua.obtain_rois(DataSB2,blobD2)
Rois3,RoisC3,NC3 = flua.obtain_rois(DataSB3,blobD3)
Rois4,RoisC4,NC4 = flua.obtain_rois(DataSB4,blobD4)

## ROIS Color Analysis

In [ ]:
ChanX='G'
ChanY='R'
Xchan1,Ychan1=flup.rois_last_frame_2chan_plt(RoisC1,ChanX,ChanY,'Serie sfGFP')

In [ ]:
Xchan2,Ychan2=flup.rois_last_frame_2chan_plt(RoisC2,ChanX,ChanY,'Serie BeRFP')

In [ ]:
Xchan3,Ychan3=flup.rois_last_frame_2chan_plt(RoisC3,ChanX,ChanY,'Serie CyOFP')

In [ ]:
Xchan4,Ychan4=flup.rois_last_frame_2chan_plt(RoisC4,ChanX,ChanY,'Serie Mix')

In [ ]:
#fit1=flua.linear_fit(Xchan1[:,0],Ychan1[:,0])
fit1=flua.linear_fit(Xchan1[:,0],Ychan1[:,0],filename="Plots/Classifier/RG_fit1.pdf")

In [ ]:
#fit2=flua.linear_fit(Xchan2[:,0],Ychan2[:,0])
fit2=flua.linear_fit(Xchan2[:,0],Ychan2[:,0],filename="Plots/Classifier/RG_fit2.pdf")

In [ ]:
#fit3=flua.linear_fit(Xchan3[:,0],Ychan3[:,0])
fit3=flua.linear_fit(Xchan3[:,0],Ychan3[:,0],filename="Plots/Classifier/RG_fit3.pdf")

In [ ]:
ClasS4,NClasS4=flua.colony_classifier([fit1,fit2,fit3],['GFP','CyOFP','BeRFP'],Xchan4,Ychan4)

In [ ]:
Xmin=0
flup.plt_lin_fit(Xmin,90000,fit1,'g')
flup.plt_lin_fit(Xmin,90000,fit2,'y')
flup.plt_lin_fit(Xmin,30000,fit3,'r')
plt.plot(NClasS4['GFP'][0],NClasS4['GFP'][1],'go')
plt.plot(NClasS4['CyOFP'][0],NClasS4['CyOFP'][1],'yo')
plt.plot(NClasS4['BeRFP'][0],NClasS4['BeRFP'][1],'ro')
plt.savefig("Plots/Classifier/Classification.pdf", transparent=True)